In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('lin_reg').getOrCreate()

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
#Spark needs this format to run ML algo on data
training = spark.read.format("libsvm").load("/FileStore/tables/sample_linear_regression_data-2.txt")
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [0]:
lr = LinearRegression(
    featuresCol='features',
    labelCol='label',
    predictionCol='prediction'
)

In [0]:
lrModel = lr.fit(training)

In [0]:
lrModel.coefficients

Out[10]: DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [0]:
lrModel.intercept

Out[11]: 0.14228558260358093

In [0]:
training_summary = lrModel.summary
print(training_summary.r2)
print(training_summary.rootMeanSquaredError)

0.027839179518600154
10.16309157133015


In [0]:
#Split data into training and test data
all_data = spark.read.format('libsvm').load("/FileStore/tables/sample_linear_regression_data-2.txt")

train_data, test_data = all_data.randomSplit([0.7,0.3])

In [0]:
train_data.describe().show()
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                335|
|   mean| 0.8579709915165366|
| stddev|  10.25017884252641|
|    min|-28.046018037776633|
|    max|  27.78383192005107|
+-------+-------------------+

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                166|
|   mean|-0.9561384472189802|
| stddev| 10.378290928397847|
|    min|-28.571478869743427|
|    max| 24.290551295953957|
+-------+-------------------+



In [0]:
correct_model = lr.fit(train_data)

In [0]:
test_results = correct_model.evaluate(test_data)

In [0]:
# test_results.residuals.show()

In [0]:
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [0]:
predictions = correct_model.transform(unlabeled_data)
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -0.6533363489129723|
|(10,[0,1,2,3,4,5,...|   2.117981583874538|
|(10,[0,1,2,3,4,5,...| -0.6743401610932666|
|(10,[0,1,2,3,4,5,...|   4.305676247069411|
|(10,[0,1,2,3,4,5,...|  0.5767032489931582|
|(10,[0,1,2,3,4,5,...|  1.8109882550389633|
|(10,[0,1,2,3,4,5,...|  0.0389405445615475|
|(10,[0,1,2,3,4,5,...| -2.8735927044239227|
|(10,[0,1,2,3,4,5,...| -0.4293605421008443|
|(10,[0,1,2,3,4,5,...| -1.8370346468962317|
|(10,[0,1,2,3,4,5,...| 0.22579525636086328|
|(10,[0,1,2,3,4,5,...|-0.23922210866187632|
|(10,[0,1,2,3,4,5,...|  0.9201420272687896|
|(10,[0,1,2,3,4,5,...|   2.154884900001976|
|(10,[0,1,2,3,4,5,...|  0.6860100586519249|
|(10,[0,1,2,3,4,5,...| -1.9646542028674974|
|(10,[0,1,2,3,4,5,...| -1.1295828785321356|
|(10,[0,1,2,3,4,5,...|  1.7431650361389424|
|(10,[0,1,2,3,4,5,...|  1.3445852126075406|
|(10,[0,1,2,3,4,5,...|   4.97660